In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
from thesis_v2 import dir_dict
from thesis_v2.training.training_aux import load_training_results

In [3]:
from sys import path
from os.path import join, exists, dirname
from os import makedirs



In [4]:
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200218 as param_iterator_obj,
    keygen
)

In [5]:
from numpy.linalg import norm
import numpy as np

In [6]:
from sklearn.decomposition import PCA
from pickle import dump

In [7]:


def calc_raw_stats(weight_matrix):
    # weight_matrix is N * D
    # N = number of filters
    # D = size of a (flattened) filter.
    mean_all = weight_matrix.mean(axis=1)
    std_all = weight_matrix.std(axis=1)
    print('mean_all', mean_all.mean(), mean_all.std())
    print('std_all', std_all.mean(), std_all.std())
    
def calc_normed_pca(weight_matrix, verbose=False):
    # here `normed` means PCA after normalization of weight_matrix
    
    # there are multiple ways to define `normalization`
    # 
    # a simple way is to make each flattened filter with unit norm.
    weight_matrix_norm = norm(weight_matrix, axis=1, keepdims=True)
#     assert weight_matrix_norm.shape == (256,1)
    weight_matrix_normed = weight_matrix/weight_matrix_norm
    
    # tested, works as expected.
    if verbose:
        assert np.allclose(norm(weight_matrix_normed, axis=1), 1.0)
    
    
    # then let's do PCA
    pca_obj = PCA(svd_solver='full')
    pca_obj.fit(weight_matrix_normed)
    if verbose:
        # top 10 explains 60%;
        # top 20 explains 85%;
        # looks fine to me, compared to https://doi.org/10.1101/677237
        # "Recurrent networks can recycle neural resources to flexibly trade speed for accuracy in visual recognition"
        
        print(np.cumsum(pca_obj.explained_variance_ratio_)[:20])
        print(pca_obj.components_.shape)
    
    # then let's return and save them.
    return {
        'weight_matrix_norm': weight_matrix_norm,
        'weight_matrix': weight_matrix,
        'components': pca_obj.components_,
        'explained_variance_ratio': pca_obj.explained_variance_ratio_,
    }

def construct_back_by_interleave(x1, x2, by1, by2):
    n1, m1 = x1.shape
    n2, m2 = x2.shape
    assert n1 % by1 == 0
    assert n2 % by2 == 0
    assert m1==m2 and n1//by1 == n2//by2
    
    x1 = x1.reshape(n1//by1, by1, m1)
    x2 = x2.reshape(n2//by2, by2, m2)
    return np.concatenate((x1,x2), axis=1).reshape((n1+n2, m1))


def save_one_result(filename, data):
    makedirs(dirname(filename), exist_ok=True)
    with open(filename, 'wb') as f:
        dump(data, f)

In [8]:
# create dir
global_vars = {
    'save_dir': join(dir_dict['analyses'],
                     'scripts+yuanyuan_8k_a_3day+maskcnn_polished_with_rcnn_k_bl+20200218',
                     'pca')
}

def prepare_dir():
    save_dir = global_vars['save_dir']
    if not exists(save_dir):
        makedirs(save_dir)
prepare_dir()

In [9]:
# compute ccmax
from strflab.stats import cc_max
from thesis_v2.data.prepared.yuanyuan_8k import get_neural_data_per_trial

cc_max_all_neurons = cc_max(get_neural_data_per_trial(
    ('042318', '043018','051018',))
                           )
assert cc_max_all_neurons.shape == (79,)

In [10]:
def collect_all():
    save_dir = global_vars['save_dir']
    count = 0
    verbose = True
    for idx, param in enumerate(param_iterator_obj().generate()):
        
        if param['rcnn_bl_cls'] <= 1:
            # ignore cls=1,2 models.
            continue
        
        key = keygen(**{k: v for k, v in param.items() if k not in {'scale', 'smoothness'}})
        result = load_training_results(key, return_model=False, return_checkpoint=True)
        cc_raw = np.asarray(result['stats_best']['stats']['test']['corr'])
        assert cc_raw.shape == (79,)
        # load loss curve to make sure we are good in approximation.
        corr_test = ((cc_raw/cc_max_all_neurons)**2).mean()
        # only keep 0.6+ models
        if corr_test < 0.6:
            continue
        
        count += 1        
        
        def filename_gen(x):
            return join(save_dir, f'{idx}/unit_norm', f'{x}.pkl')
        
        filename_set = {'ff', 'lateral'}
        if all([exists(filename_gen(x)) for x in filename_set]):
            # right now if partially done, the partial results will be fully overwritten.
            continue
        
        # check layer one lateral.
        weight_np_ff = result['checkpoint']['model']['moduledict.bl_stack.layer_list.0.b_conv.weight'].numpy().astype(np.float64)
        weight_np_lateral = result['checkpoint']['model']['moduledict.bl_stack.layer_list.0.l_conv.weight'].numpy().astype(np.float64)

        kernel_shape = weight_np_ff.shape[2:]
        assert weight_np_ff.shape[2:] == weight_np_lateral.shape[2:] == kernel_shape
        assert weight_np_ff.ndim == weight_np_lateral.ndim == 4
        channel_low = weight_np_ff.shape[1]
        channel_high = weight_np_ff.shape[0]
        assert weight_np_lateral.shape[:2] == (channel_high, channel_high)
        
        weight_np_ff = weight_np_ff.reshape((channel_low*channel_high, kernel_shape[0]*kernel_shape[1]))
        weight_np_lateral = weight_np_lateral.reshape((channel_high*channel_high, kernel_shape[0]*kernel_shape[1]))
        
        
#         print(weight_np_lateral.shape, weight_np_ff.shape)
        
        if verbose:
            print('idx', idx)
            print('key', key)
            # show some raw stats.
            calc_raw_stats(weight_np_ff)
            calc_raw_stats(weight_np_lateral)
        pca_res_dict = dict()
        pca_res_dict['ff'] = calc_normed_pca(weight_np_ff, verbose)
        pca_res_dict['lateral'] = calc_normed_pca(weight_np_lateral, verbose)
        
        for fn in filename_set:
            save_one_result(filename_gen(fn), pca_res_dict[fn])
        
    print(count)


In [11]:
collect_all()

idx 4
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.001233146453766191 0.014254920461633095
std_all 0.05776098614490341 0.030424982758348776
mean_all -0.006143061898740145 0.04943630172530213
std_all 0.05666960729290334 0.029920704763095576
[0.32765151 0.54351898 0.67840821 0.75973898 0.83582164 0.89586914
 0.93699505 0.97179571 1.        ]
(9, 9)
[0.31219754 0.53170469 0.67547052 0.80746448 0.8862151  0.95445517
 0.98168075 0.99523403 1.        ]
(9, 9)
idx 5
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.00027818103257162526 0.01533423528532285
std_all 0.05866368892425136 0.029065552086673076
mean_all -0.00527112569392

[0.20506642 0.35767509 0.49621655 0.62047483 0.69362708 0.75469244
 0.80277413 0.84701478 0.87717125 0.9028267  0.92172768 0.93860618
 0.95112458 0.96203544 0.97094089 0.97676447 0.98217477 0.98602325
 0.98907469 0.991786  ]
(25, 25)
idx 43
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/k_l235/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.0005528622007318518 0.008191655882412876
std_all 0.047980548318581465 0.03005918025927135
mean_all -0.007400256077148698 0.025922041172888547
std_all 0.05989763329387811 0.0191644532652592
[0.14881371 0.26527899 0.36992805 0.46467855 0.545337   0.61426243
 0.67561707 0.73089862 0.77829025 0.81456824 0.84864551 0.87386933
 0.89613528 0.91412919 0.92957584 0.9431124  0.95568293 0.96693997
 0.97668246 0.98275509]
(25, 25)
[0.15606502 0.29011843 0.40192231 0.4927304  0.57579156 0.65018239
 0.70947957 0.75800713 0.7987

idx 66
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/sc0.01/sm0.000005/lmse/m_se0
mean_all 0.0005550239513280413 0.013821685427760301
std_all 0.0496265068093843 0.03176815893716944
mean_all -0.0037894611531344123 0.0458961501507386
std_all 0.05292637653591957 0.02126636418218453
[0.30122063 0.4704407  0.63105528 0.7404969  0.83480472 0.88716714
 0.92895068 0.96816586 1.        ]
(9, 9)
[0.27953618 0.51315148 0.66686676 0.79205584 0.87184106 0.93980197
 0.96884755 0.9881003  1.        ]
(9, 9)
idx 67
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.0006810269590917053 0.016696174013979666
std_all 0.05390764472242629 0.03133198073009381
mean_all -0.00609909103454

idx 85
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/k_l237/sc0.01/sm0.000005/lmse/m_se0
mean_all 0.00048650933112867 0.00899531815420864
std_all 0.0443216906671398 0.02588725965907107
mean_all -0.0017367748584151098 0.023780968552969604
std_all 0.04916753634331052 0.018547406381312435
[0.15579625 0.26400988 0.36035049 0.43829387 0.49669074 0.54530673
 0.58355566 0.61683896 0.64742174 0.67724131 0.70265556 0.7271422
 0.74905998 0.77067392 0.78963103 0.80753259 0.82398003 0.83922074
 0.85227162 0.86486028]
(49, 49)
[0.19658801 0.33471395 0.41765548 0.49251148 0.55813783 0.6172768
 0.67064307 0.70964615 0.74704872 0.77949895 0.80697442 0.83267413
 0.85373969 0.8674061  0.87995843 0.89100647 0.90186628 0.9122853
 0.92216776 0.93124555]
(49, 49)
idx 86
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l2/k_l1

idx 103
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.0008153642524716769 0.01311756417471104
std_all 0.03765178303606415 0.02521378801961138
mean_all -0.005532660609998847 0.03358384676905129
std_all 0.038972261902369895 0.01624601937728945
[0.24508802 0.42010371 0.58620012 0.69855566 0.79647946 0.8857508
 0.93344447 0.9718291  1.        ]
(9, 9)
[0.31293811 0.52753074 0.69152034 0.79052681 0.86774187 0.92493249
 0.95711423 0.98218347 1.        ]
(9, 9)
idx 106
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l235/sc0.01/sm0.000005/lmse/m_se0
mean_all 0.00017765764430336634 0.009056097392074412
std_all 0.03624706872581644 0.02052622420248823
mean_all -0.0047

...

[0.28425762 0.48377676 0.62649721 0.75460162 0.84254635 0.92261236
 0.95481195 0.98445776 1.        ]
(9, 9)
idx 1475
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/sc0.01/sm0.000005/lpoisson/m_se1
mean_all -0.000688080330930004 0.018071898392904854
std_all 0.0504496862211956 0.03743633135358218
mean_all -0.0019969482380102956 0.06366703714655629
std_all 0.0559191776651666 0.027944860241467948
[0.30870933 0.52304057 0.64511001 0.76565882 0.83973382 0.90296006
 0.94160577 0.97931362 1.        ]
(9, 9)
[0.3367555  0.53963475 0.67433242 0.79111552 0.87379221 0.94245854
 0.9663587  0.98856377 1.        ]
(9, 9)
idx 1476
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/sc0.01/sm0.000005/lpoisson/m_s

idx 1490
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l237/sc0.01/sm0.000005/lpoisson/m_se1
mean_all 0.0004711811852534142 0.008339616454316053
std_all 0.044562783757857695 0.03491690099580076
mean_all 6.391701467327635e-05 0.03780811092047314
std_all 0.06371518846342816 0.024436388246907734
[0.20892083 0.31899115 0.40593312 0.48479523 0.54836609 0.58729119
 0.62369473 0.65689538 0.68660162 0.71184385 0.73250731 0.75207651
 0.77074394 0.78655158 0.80091371 0.81469951 0.82747312 0.84003804
 0.8507848  0.86041591]
(49, 49)
[0.17000789 0.28605388 0.37605388 0.44918977 0.51671742 0.57026196
 0.61751376 0.65985488 0.70127759 0.73668592 0.76873745 0.79745225
 0.81807176 0.83720151 0.85555385 0.86910727 0.88235306 0.89411797
 0.90407138 0.91294648]
(49, 49)
idx 1491
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out

mean_all -0.00025029149644681186 0.00857387239317215
std_all 0.03739526115912321 0.022325205669436
mean_all -0.00014163595255967232 0.03307350467850118
std_all 0.04366649118997534 0.01994473328621812
[0.2817294  0.37509698 0.44572835 0.51012593 0.55795253 0.59374109
 0.62714236 0.65932377 0.68513615 0.70775057 0.72971451 0.74779474
 0.76415816 0.77863673 0.7902979  0.80085245 0.81091584 0.82064453
 0.82997827 0.83814477]
(81, 81)
[0.26447539 0.40145188 0.50786185 0.57016922 0.62591734 0.66731273
 0.70701289 0.74398716 0.7750781  0.80514676 0.82991328 0.85049259
 0.86648723 0.87945165 0.88882791 0.8973113  0.90486633 0.91181985
 0.91832138 0.92386685]
(81, 81)
idx 1503
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/k_l239/sc0.01/sm0.000005/lpoisson/m_se1
mean_all -8.920648286432658e-05 0.009335658809884516
std_all 0.03800484809516322 0.023

mean_all 0.0004667976448065092 0.011742350357558889
std_all 0.033066871655940806 0.018407942091989933
mean_all -0.0042349496317831935 0.031041631730881907
std_all 0.03395248170996722 0.016653773648167128
[0.17309298 0.33816882 0.44824386 0.54743988 0.63252785 0.69567351
 0.74041566 0.77618783 0.8065177  0.83424553 0.85814041 0.87596017
 0.89254654 0.90622274 0.9196817  0.93113195 0.94169023 0.95085318
 0.95938872 0.96745041]
(25, 25)
[0.3231188  0.47600084 0.6025772  0.7107837  0.77056543 0.82188906
 0.84746865 0.87034615 0.88837691 0.9056146  0.91830883 0.92905159
 0.93811785 0.94680295 0.95428831 0.96033986 0.96589357 0.97104777
 0.97598425 0.98059705]
(25, 25)
idx 1522
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l237/sc0.01/sm0.000005/lpoisson/m_se1
mean_all 5.775187445044631e-05 0.010559226290049255
std_all 0.03454741334187504 0.0

idx 1533
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/k_l239/sc0.01/sm0.000005/lpoisson/m_se1
mean_all 3.28785122258687e-05 0.010482274593093662
std_all 0.03503211206651588 0.019604453025058057
mean_all -0.0033416169306743023 0.029265644751637704
std_all 0.03844440162885204 0.01594001726268222
[0.20110101 0.30913073 0.40318632 0.48159319 0.53530517 0.58679694
 0.62860215 0.65661159 0.68330075 0.70893546 0.73029454 0.74909818
 0.76721386 0.78211147 0.79503981 0.80723732 0.81724776 0.82664602
 0.83564863 0.84387087]
(81, 81)
[0.26254915 0.39993756 0.51668703 0.60607961 0.66879853 0.71585082
 0.75265465 0.78373474 0.81029373 0.83228012 0.84891624 0.86281044
 0.8758997  0.88572518 0.89351307 0.90096748 0.90754899 0.91388895
 0.91873114 0.92313231]
(81, 81)
idx 1534
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/ou